In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

In [2]:
import numpy as np;
import pandas as pd;

In [ ]:
multi_data = pd.read_csv('../data/zri_multifamily_v2.csv', index_col = 0);

In [ ]:
ZONE = 1013;
SPLIT = 60; # 2014-2018: training, 2019: testing.
BATCH_SIZE = 8;
WINDOW_SIZE = 12;

In [ ]:
single_city_data = multi_data[multi_data.index == ZONE];

In [ ]:
single_city_series = np.array(single_city_data["zri"])

In [ ]:
single_city_train = single_city_series[:SPLIT];
single_city_test = single_city_series[SPLIT:];

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
temp = windowed_dataset(single_city_train, WINDOW_SIZE, BATCH_SIZE, 4)

In [3]:
NN_1_pd = pd.read_csv('./NN_1.csv', index_col = 0);

In [4]:
for i in range(2,3):
    temp = pd.read_csv('./NN_{}.csv'.format(i), index_col = 0);
    NN_1_pd = pd.concat([NN_1_pd, temp], axis = 0);

In [5]:
NN_1_pd.describe()

,RMSE_train,RMSE_test,RMSE_pure
count,1302.000000,1302.000000,1302.000000
mean,7.278545,52.037030,63.325435
std,4.370064,44.959590,51.305926
min,0.835454,3.366397,3.651505
25%,4.213621,23.185900,28.657435
50%,6.210552,37.476439,46.773726
75%,9.243207,64.001536,81.726586
max,42.022008,379.465871,372.691826


In [6]:
NN_1_pd.to_csv('NN_1_window_12_maxoverfit_63.33.csv', index = False)

In [15]:
NN_1_pd.sort_values("RMSE_pure", ascending = False, inplace = True)

In [16]:
NN_1_pd.head(50)

,RMSE_train,RMSE_test,RMSE_pure,forecast
zip,,,,
95928,9.932908,930.597307,598.389780,[1348.3163 1053.8423 948.82556 952.66345 ...
90034,14.356650,37.939498,567.081056,[2616.6743 2546.6365 2442.3362 2321.5083 2213....
33706,9.735983,154.839692,465.738396,[1519.8743 1519.5657 1500.925 1459.8 1...
92123,13.247541,71.915119,446.547315,[2252.7734 2196.53 2103.878 1979.1663 1865....
76006,7.423790,343.313974,412.294219,[1019.4907 959.1782 849.57935 717.42944 ...
90270,22.482006,122.094585,391.641137,[2235.8696 2264.1233 2300.391 2333.5276 2347....
90640,15.991424,133.495006,366.912427,[2153.878 2192.2546 2255.3494 2288.2502 2319....
90255,23.222581,96.076967,359.685381,[2204.182 2047.5607 2025.5354 2164.1418 2298....
11542,8.119495,75.521963,355.563804,[2338.3445 2314.8718 2279.0664 2229.8472 2168....
